In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/My\ Drive/Skin\ Cancer\ Prediction/skin-cancer-malignant-vs-benign

/content/drive/My Drive/Skin Cancer Prediction/skin-cancer-malignant-vs-benign


In [4]:
!nvidia-smi

Mon Sep 14 03:48:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
#from keras.layers.normalization import BatchNormalization

In [6]:
# Initialising the CNN
classifier = Sequential()

In [7]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation='relu'))
#classifier.add(BatchNormalization())

In [8]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [9]:
classifier.add(Dropout(0.2))

In [10]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [11]:
classifier.add(Dropout(0.4))

In [12]:
# Step 3 - Flattening
classifier.add(Flatten())

In [13]:
# Step 4 - Full connection
classifier.add(Dense(units = 256, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [14]:
from tensorflow import keras
adam = keras.optimizers.Adam(learning_rate=0.0001)

In [15]:
# Compiling the CNN'
classifier.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0

In [17]:
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
train_datagen

In [20]:
training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size = (128, 128),
                                                 batch_size = 16,
                                                 class_mode = 'binary')

Found 2395 images belonging to 2 classes.


In [21]:
len(training_set)

150

In [22]:
test_set = test_datagen.flow_from_directory('data/test',
                                            target_size = (128, 128),
                                            batch_size = 16,
                                            class_mode = 'binary')

Found 600 images belonging to 2 classes.


In [23]:
len(test_set)

38

In [25]:
model = classifier.fit_generator(training_set,
                       steps_per_epoch = len(training_set),
                       epochs = 50,
                       validation_data = test_set,
                       validation_steps = len(test_set))

Epoch 1/50
150/150 [==============================] - 57s 380ms/step - loss: 0.5149 - accuracy: 0.7395 - val_loss: 0.5631 - val_accuracy: 0.7417
Epoch 2/50
150/150 [==============================] - 17s 112ms/step - loss: 0.4688 - accuracy: 0.7582 - val_loss: 0.4749 - val_accuracy: 0.8000
Epoch 3/50
150/150 [==============================] - 17s 113ms/step - loss: 0.4550 - accuracy: 0.7745 - val_loss: 0.4769 - val_accuracy: 0.7983
Epoch 4/50
150/150 [==============================] - 17s 113ms/step - loss: 0.4264 - accuracy: 0.7900 - val_loss: 0.4637 - val_accuracy: 0.8000
Epoch 5/50
150/150 [==============================] - 17s 113ms/step - loss: 0.4244 - accuracy: 0.7942 - val_loss: 0.4568 - val_accuracy: 0.8017
Epoch 6/50
150/150 [==============================] - 17s 112ms/step - loss: 0.4075 - accuracy: 0.7958 - val_loss: 0.4487 - val_accuracy: 0.8167
Epoch 7/50
150/150 [==============================] - 17s 112ms/step - loss: 0.4006 - accuracy: 0.8100 - val_loss: 0.4411 - val_ac

In [31]:
from tensorflow.keras.models import load_model
classifier.save('model_benign_malignant_50epochs.h5')

# Single Prediction

In [26]:
import numpy as np
from keras.preprocessing import image

In [27]:
MODEL_PATH ='model_benign_malignant_50epochs.h5'

In [28]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [32]:
from keras.models import load_model
classifier = load_model(MODEL_PATH)

In [33]:
training_set.class_indices

{'benign': 0, 'malignant': 1}

In [34]:
test_image = image.load_img('malignant_single_pred.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
if result[0][0] == 1:
  prediction = 'malignant'
else:
  prediction = 'benign'
print(result)

[[1.]]


In [42]:
test_image = image.load_img('benign_single_pred.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
if result[0][0] == 1:
  prediction = 'malignant'
else:
  prediction = 'benign'
print(result)

[[0.]]
